# Using Landlab and PyGMT Together to Create Interactive Erosion Slider

As we have seen, we can use Landlab and PyGMT together to download data, convert to and from PyGMT to Landlab, and  visualize our output using PyGMT plotting functions. Expanding on our usage of Landlab and PyGMT, in this tutorial we will explore:

- PyGMT Visualizations:
    - DEM data visualized with grdimage for realistic mapping. 
    
- Flow Path Overlay:
    - Threshold-based flow path extraction, visualized with <code>pygmt.plot</code>.
    
- Interactive Modeling:
    - Users can adjust the erodibility factor (K_sp) and observe terrain evolution in real-time with PyGMT maps.

## Required packages:
- Numpy
- PyGMT
- Landlab

We will begin by importing the following packages

In [ ]:
import numpy as np
import pygmt
from landlab import RasterModelGrid
from landlab.components import FlowAccumulator, DepressionFinderAndRouter
from landlab.plot import imshow_grid
from ipywidgets import interact, FloatSlider

We will begin using the <code>RasterModelGrid</code> function from Landlab to create our grid

In [ ]:
#Define grid size and node spacing
ncols, nrows = 100, 100
dx = 0.01
#Create grid object
mg = RasterModelGrid((nrows, ncols), dx)

Then, loading DEM data from PyGMT. In this example we will use topographic elevation data.

In [ ]:
#Load DEM data
z = mg.add_field("topographic__elevation", np.random.rand(nrows, ncols) * 1000, at="node")

To make the terrain more realistic, we will use <code>np.random.rand</code> to apply random pertubations to our data

In [ ]:
z += np.random.rand(nrows, ncols) * 50

We can visualize this data by using <code>Figure</code> and <code>grdimage</code>

In [ ]:
#Visualize the initial terrain using PyGMT
fig = pygmt.Figure()
fig.grdimage(grid=z.reshape(nrows, ncols), region=[0, dx * ncols, 0, dx * nrows],
             projection="X10c/10c", #Cartesion projection
             cmap="viridis", frame=["a", "WSen+tInitial Topography"])
fig.colorbar(frame="af+lElevation (m)")
fig.show()

Next, we will create the elements we need to visualize the drainage area

In [ ]:
#Initialize flow routing and depression finder
fa = FlowAccumulator(mg)
df = DepressionFinderAndRouter(mg)

#Run flow routing and depression filling
fa.run_one_step()
df.map_depressions()

Extracting drainage and elevation area and reshaping them into 2D array

In [ ]:
#Extract flow accumulation and mask depressions
drainage_area = mg.at_node["drainage_area"].reshape(nrows, ncols)
filled_elevation = mg.at_node["topographic__elevation"].reshape(nrows, ncols)

Now, visualizing drainage area

In [ ]:
#Visualize drainage area with PyGMT
fig = pygmt.Figure()
fig.grdimage(grid=drainage_area, region=[0, dx * ncols, 0, dx * nrows], projection="X10c/10c",
            cmap="plasma", frame=["a", "WSen+tDrainage Area"])
fig.colorbar(frame="af+lArea (m²)")
fig.show()

In [ ]:
#Step 3: Overlaying Flow Paths with PyGMT
#Define top 5% of drainage area, ie. the major streams
threshold = np.percentile(drainage_area, 95)
stream_nodes = mg.at_node["drainage_area"] > threshold

#Extract node coordinates for flow paths
stream_x, stream_y = mg.node_x[stream_nodes], mg.node_y[stream_nodes]

#Visualize flow paths on topography
fig = pygmt.Figure()
fig.grdimage(grid=filled_elevation, region=[0, dx * ncols, 0, dx * nrows], projection="X10c/10c",
             cmap="viridis", frame=["a", "WSen+tFlow Paths on Topography"])
fig.plot(x=stream_x * dx, y=stream_y * dx, style="c0.1c", color="blue", label="Flow Paths")
fig.legend()
fig.colorbar(frame="af+lElevation (m)")
fig.show()

For interactive modeling, we will define our function <code>run_erosion</code>. Passing through the erodability factor <code>K_sp</code>, this function will provide us visualization of different values of <code>K_sp</code> on erosion.

In [ ]:
def run_erosion(K_sp):
    #Reset DEM
    mg.at_node["topographic__elevation"][:] = z.copy()

    #Define constants
    uplift_rate = 0.01 #Tectonic lift
    n_steps = 5 #steps
    
    for _ in range(n_steps):
        mg.at_node["topographic__elevation"][mg.core_nodes] += uplift_rate
        fa.run_one_step()
        mg.at_node["topographic__elevation"][mg.core_nodes] -= K_sp * mg.at_node["drainage_area"][mg.core_nodes]

    #Plot eroded terrain using PyGMT
    eroded_elevation = mg.at_node["topographic__elevation"].reshape(nrows, ncols)
    fig = pygmt.Figure()
    fig.grdimage(grid=eroded_elevation, region=[0, dx * ncols, 0, dx * nrows], projection="X10c/10c",
                 cmap="viridis", frame=["a", f"WSen+tEroded Topography (K_sp={K_sp})"])
    fig.colorbar(frame="af+lElevation (m)")
    fig.show()

Run <code>run_ersion</code> and visualize results 

In [ ]:
interact(run_erosion, K_sp=FloatSlider(value=1e-5, min=1e-6, max=1e-4, step=1e-6, description="K_sp"))